In [3]:
import sklearn
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.datasets import make_swiss_roll
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde

In [4]:
import torchvision
batch_size = 256
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('datasets/MNIST/train.data', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('datasets/MNIST/test.data', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=True)

/home/cmackinnon/scripts/py394/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from HPO.searchspaces.DARTS_NAS_config import init_config
cs = init_config()
results_df = pd.DataFrame(columns = ["labels","prediction","risk","augmented","ID"])

In [6]:
from HPO.utils.model import NetworkMain
from HPO.utils.DARTS_utils import config_space_2_DARTS
hp = cs.sample_configuration().get_dictionary()
gen = config_space_2_DARTS(hp,reduction = True)
model = NetworkMain(50,32,num_classes= 2, 
                      layers = 3, auxiliary = False,drop_prob = 0, genotype = gen, binary = True)

In [7]:
params = 0
for parameter in model.state_dict():
    print(parameter)
    print(model.state_dict()[parameter].shape)
    params += np.prod(model.state_dict()[parameter].shape)
    print(params)

    

stem.0.weight
torch.Size([96, 50, 3])
14400
stem.1.weight
torch.Size([96])
14496
stem.1.bias
torch.Size([96])
14592
stem.1.running_mean
torch.Size([96])
14688
stem.1.running_var
torch.Size([96])
14784
stem.1.num_batches_tracked
torch.Size([])
14785.0
cells.0.preprocess0.op.1.weight
torch.Size([64, 96, 1])
20929.0
cells.0.preprocess0.op.2.weight
torch.Size([64])
20993.0
cells.0.preprocess0.op.2.bias
torch.Size([64])
21057.0
cells.0.preprocess0.op.2.running_mean
torch.Size([64])
21121.0
cells.0.preprocess0.op.2.running_var
torch.Size([64])
21185.0
cells.0.preprocess0.op.2.num_batches_tracked
torch.Size([])
21186.0
cells.0.preprocess1.op.1.weight
torch.Size([64, 96, 1])
27330.0
cells.0.preprocess1.op.2.weight
torch.Size([64])
27394.0
cells.0.preprocess1.op.2.bias
torch.Size([64])
27458.0
cells.0.preprocess1.op.2.running_mean
torch.Size([64])
27522.0
cells.0.preprocess1.op.2.running_var
torch.Size([64])
27586.0
cells.0.preprocess1.op.2.num_batches_tracked
torch.Size([])
27587.0
cells.0._op

In [12]:
def convert_weight_shape(model):
    inverse_dict = {}
    flattened = []
    for parameter in model.state_dict():
        inverse_dict[parameter] = model.state_dict()[parameter].shape
        flattened.append(np.reshape(model.state_dict()[parameter],(1,-1)))
    return flattened,inverse_dict

In [13]:

t, inv= convert_weight_shape(model)

In [21]:
weights = torch.cat(t,dim = 1)
weight_np = np.asarray(weights)
weight_np.shape

(1, 259487)

In [28]:
from KDEpy import FFTKDE
X,y =FFTKDE(bw=0.001).fit(weight_np).evaluate(10)

ValueError: maximum supported dimension for an ndarray is 32, found 259488